# Run Your Pool - Seed Optimization

The goal of this notebook is to determine the optimal seed combination to win a `points per win * seed` style pool. Off the cuff, I thought about just using the overall historical win percentage by round for each seed to calculate an expected value of wins by each seed, multiply that by the seed number, sum them all up and call it a day. But then it occurred to me that doesn't account for __when__ these teams play each other: we're picking 16 teams, so you're looking for the highest expected value for a combination of teams, not necessarily the highest combination of expected values (these might be equal with independent events, but these events are obviously not independent).

The methodology is simple: I scraped all available seed vs. seed historical win percentages, and these will serve as our matchup win probabilities. We'll simulate each region N thousand times, recording the total points by seed for each iteration, and then we'll find which 4 seed combination has the highest expected total points.

### Limitations and Assumptions

1. __Win probabilities are only considered from the modern era of the tournament: 1985-present__. This means that the we only have 38 tournaments to use when generating historical win percentages. Because of this limited sample size, there are two problems that come up:

    1. __Not every possible matchup has been played__: There are a number of matchup combinations that have never been played, and some of them will surprise you (for example, a 5 seed has never played a 7 seed). However, if we run are running tens of thousands of simulations, these matchups are going to eventually happen. In these cases, the surrogate win probability will be the average of all other matchup probabilities with the same seed differential. There is one case where no matchup has ever been played with a seed differential of 14 (a 15 has never played a 1, and a 16 has never played a 2), so an arbitrary 90% will be used in that case.
    
    2. __Infrequent matchups have unrealistic probabilities__: Because some matchups occur infrequently, the win probability is unlikely to be representative of the true distribution. For example: 7 seeds are 0-4 all time against 11 seeds, meaning that the win probability for a potential 7-11 matchup in this experiment would be 0% for a 7 seed to beat a 11 seed. This obviously ridiculous, but given that we're doing this exercise 2 days before the tournament starts I'm not sure I'll have time to come up with an alternative plan. The hope with this limitation and the one above is that since these matchups occur infrequently, sufficiently high sample size will still converge to the true expected value even with some fluky probabilities. 
    
2. __Only simulating 1 region__: All head to head probabilities in this notebook are from within a single region for seeds 1-16. The assumption is that the optimal configuration of 16 teams is to pick 4 from each region, which allows for the possibility of two wins per team before they play each other. We also assume that there is one highest expected possible point combination, and thus you should always select the same seed combinations in each region. This also means that the Final Four / Championship are not being simulated, so they are not included in this analysis. 


If you can live with these limitations, then read on.

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
from itertools import combinations

import json
import numpy as np
import pandas as pd

import data_scraping_utils as dutils
import optimization_utils as opt_utils

<IPython.core.display.Javascript object>

### Start by scraping head-to-head matchup historical win percentage from wikipedia: https://en.wikipedia.org/wiki/NCAA_Division_I_men%27s_basketball_tournament#Seed_pairing_results

In [4]:
win_probability_dictionary = dutils.scrape_winning_percentage_dict_from_wikipedia()
# display dictionary
win_probability_dictionary

{1: {16: 0.987,
  8: 0.784,
  9: 0.921,
  4: 0.719,
  5: 0.766,
  12: 1.0,
  13: 1.0,
  2: 0.489,
  3: 0.615,
  6: 0.8,
  7: 1.0,
  10: 0.833,
  11: 0.5,
  14: -1.0,
  15: -1.0},
 2: {15: 0.928, 7: 0.69, 10: 0.648, 3: 0.625, 6: 0.793, 11: 0.833, 14: -1.0},
 3: {14: 0.855, 6: 0.615, 11: 0.615},
 4: {13: 0.789,
  5: 0.557,
  12: 0.683,
  2: 0.571,
  3: 0.6,
  6: 0.667,
  7: 0.4,
  10: 1.0,
  11: -1.0,
  14: -1.0,
  15: -1.0},
 5: {12: 0.651,
  2: 0.8,
  3: 0.333,
  6: 1.0,
  7: -1.0,
  10: 1.0,
  11: -1.0,
  14: -1.0,
  15: -1.0},
 6: {11: 0.618},
 7: {10: 0.612, 3: 0.375, 6: 0.375, 11: 0.0, 14: 1.0},
 8: {9: 0.487,
  4: 0.545,
  5: 1.0,
  12: 0.0,
  13: 1.0,
  2: 0.6,
  3: 0.0,
  6: 1.0,
  7: 1.0,
  10: -1.0,
  11: -1.0,
  14: -1.0,
  15: 1.0},
 16: {8: -1.0,
  9: 0.0,
  4: -1.0,
  5: -1.0,
  12: -1.0,
  13: -1.0,
  2: -1.0,
  3: -1.0,
  6: -1.0,
  7: -1.0,
  10: -1.0,
  11: -1.0,
  14: -1.0,
  15: -1.0},
 15: {7: 0.667, 10: 0.0, 3: 0.333, 6: 0.0, 11: -1.0, 14: -1.0},
 14: {6: 0.125, 11

<IPython.core.display.Javascript object>

### Many of these matchups have never occurred - so we'll create a distribution of average win percentage by seed difference, and use this to generate surrogate probabilities

In [7]:
win_probability_dictionary = dutils.add_surrogate_win_probabilities_for_matchups(
    win_probability_dictionary=win_probability_dictionary,
)

<IPython.core.display.Javascript object>

##### Store dictionary so we don't have to scrape it again

In [9]:
with open("head_to_head_win_probability_dictionary.json", "w") as f:
    json.dump(win_probability_dictionary, f)

<IPython.core.display.Javascript object>

### Simulate 10,000 tournaments using this win probability dictionary and look at highest average points by seed.

In [10]:
tourney_results = []
for _ in range(10000):
    region = opt_utils.RegionBracket(
        seed_path_dictionary=opt_utils.SEED_PATH_DICTIONARY,
        win_probability_dictionary=win_probability_dictionary,
    )
    region.simulate_region_tournament()
    tourney_results.append(region.points_by_seed)
tourney_result_df = pd.DataFrame(tourney_results)

<IPython.core.display.Javascript object>

In [11]:
average_points_added_by_seed = tourney_result_df.mean().sort_values(ascending=False)

<IPython.core.display.Javascript object>

In [12]:
average_points_added_by_seed

11    7.3689
12    6.2436
6     6.2022
7     6.1768
4     6.0376
10    5.9840
8     5.6168
5     5.5415
9     5.3649
3     5.2176
2     4.4026
13    3.1915
1     2.9055
14    2.2666
15    1.6635
16    0.2416
dtype: float64

<IPython.core.display.Javascript object>

#### Key Takeaways
1. The 11 seeds are the highest expected added value by large margin, the only clear takeaway from this result is that all four 11 seeds in the tournament should be selected. 
2. Two of the top 3 expected point values is the `6/11` matchup. If you simply select the top 4 expected values from this result, you're guaranteeing that one of your teams is out after the first round. Thus, the top 4 may have the highest expected values, but there may be a more optimal configuration than simply selecting the top 4.

### What is the optimal combo?

Because the search space is discrete (and small) there's no need for a bayesian optimization search, we can just grid search this. 

In [13]:
all_combinations = list(combinations(list(np.arange(1, 17)), 4))

<IPython.core.display.Javascript object>

In [14]:
all_results = {}
for n, seed_list in enumerate(all_combinations):
    res = opt_utils.calculate_total_points_for_seed_combination(
        seed_list,
        win_probability_dictionary,
        10000,
    )
    all_results[seed_list] = res
    if n % 100 == 0:
        print(f"Completed {n} iterations")
        print("-" * 100)

Completed 0 iterations
----------------------------------------------------------------------------------------------------
Completed 100 iterations
----------------------------------------------------------------------------------------------------
Completed 200 iterations
----------------------------------------------------------------------------------------------------
Completed 300 iterations
----------------------------------------------------------------------------------------------------
Completed 400 iterations
----------------------------------------------------------------------------------------------------
Completed 500 iterations
----------------------------------------------------------------------------------------------------
Completed 600 iterations
----------------------------------------------------------------------------------------------------
Completed 700 iterations
-----------------------------------------------------------------------------------------------

<IPython.core.display.Javascript object>

In [15]:
output_dataframe = pd.DataFrame.from_dict(
    all_results, orient="index", columns=["expected_point_value"]
)

<IPython.core.display.Javascript object>

In [16]:
output_dataframe["expected_point_value"].idxmax()

(6, 7, 10, 11)

<IPython.core.display.Javascript object>

In [17]:
output_dataframe.sort_values(by=["expected_point_value"], ascending=False).iloc[:20]

,expected_point_value
"(6, 7, 10, 11)",25.8184
"(6, 10, 11, 12)",25.8056
"(6, 7, 11, 12)",25.7478
"(4, 10, 11, 12)",25.7398
"(4, 6, 11, 12)",25.7048
"(4, 6, 10, 11)",25.6783
"(4, 7, 11, 12)",25.6409
"(4, 6, 7, 11)",25.6378
"(7, 10, 11, 12)",25.5731
"(4, 7, 10, 11)",25.5156


<IPython.core.display.Javascript object>